In [46]:
# import packages
import geopandas as gpd
import osmnx as ox
import momepy
import pandana as pdna
import pandas as pd
import numpy as np
import networkx as nx


In [47]:
barriers = gpd.read_file(r"C:\Users\b8008458\OneDrive - Newcastle University\2021 to 2022\Dissertation\Github\BarrierRemoval\Study Areas\Pembrokeshire\Barriers\PembrokeBarriers.gpkg")

barriers_access = gpd.read_file(r"C:\Users\b8008458\OneDrive - Newcastle University\2021 to 2022\Dissertation\Github\BarrierRemoval\Study Areas\Pembrokeshire\Accessablility Measures\PembrokeAccessBarriersScored.gpkg")
barriers_access_bike = gpd.read_file(r"C:\Users\b8008458\OneDrive - Newcastle University\2021 to 2022\Dissertation\Github\BarrierRemoval\Study Areas\Pembrokeshire\Accessablility Measures\PembrokeAccessBarriersScoredBike.gpkg")

route_useage = gpd.read_file(r"C:\Users\b8008458\OneDrive - Newcastle University\2021 to 2022\Dissertation\Github\BarrierRemoval\Study Areas\Pembrokeshire\Bike Network\Pem_bike_network_usage.shp")
betweeness = gpd.read_file(r"C:\Users\b8008458\OneDrive - Newcastle University\2021 to 2022\Dissertation\Github\BarrierRemoval\Study Areas\Pembrokeshire\Bike Network\bike_network_costs_bc.shp")

IMD = gpd.read_file(r"C:\Users\b8008458\OneDrive - Newcastle University\2021 to 2022\Dissertation\Github\BarrierRemoval\Study Areas\Pembrokeshire\IMD\PembrokeshireIMD.shp")
carownership = "nan"

min_distance_gained = gpd.read_file(r"C:\Users\b8008458\OneDrive - Newcastle University\2021 to 2022\Dissertation\Github\BarrierRemoval\Study Areas\Pembrokeshire\ODMatrix\PembrokeODMatrix.shp")

In [48]:
# set crs
barriers = barriers.to_crs(27700)
barriers_access = barriers_access.to_crs(27700)
barriers_access_bike = barriers_access_bike.to_crs(27700)
route_useage = route_useage.to_crs(27700)
betweeness = betweeness.to_crs(27700)
IMD = IMD.to_crs(27700)
#carownership = carownership.to_crs(27700)
min_distance_gained = min_distance_gained.to_crs(27700)

In [49]:
# calculate min distances from OD Matrix

# remove i - i and j - j 
QGIS_OD_Matrix = min_distance_gained[min_distance_gained['origin_id'] != min_distance_gained['destinatio']]

# calculate minium network distance 
min_network_distances_df = QGIS_OD_Matrix.groupby('origin_id')['total_cost'].min()
# convert to dataframe
min_network_distances_df = min_network_distances_df.to_frame().reset_index() 
# join minimum distances to barriers
barriers['origin_id'] = barriers['globalid']

join = barriers.merge(min_network_distances_df, on = 'origin_id', how='left', suffixes=('', '_y'))
join.drop(join.filter(regex='_y$').columns, axis=1, inplace=True)
barriers = join
barriers = barriers.rename(columns = {'total_cost':'min_gain'})

In [51]:
# get list of current column headers
barriers_columnlist = list(barriers)

In [54]:
# join IMD rank to barrier
join_left_df = barriers.sjoin(IMD, how="left")
barriers_columnlist += ['CarPerCit'] # CarsPerCit, change to CarPerCit for Wales
barriers_columnlist += ['rank'] # IMDScore, change to "rank" for Wales
barriers = join_left_df[barriers_columnlist]


In [55]:
# join barrier_access

join = barriers.merge(barriers_access, on = 'globalid', how ='left', suffixes=('','_y'))
join.drop(join.filter(regex='_y$').columns, axis=1, inplace=True)
barriers = join

In [56]:
# join barrier_access_bike

join = barriers.merge(barriers_access_bike, on = 'globalid', how ='left', suffixes=('','_y'))
join.drop(join.filter(regex='_y$').columns, axis=1, inplace=True)
barriers = join

In [57]:
# join route_access

join = barriers.merge(barriers_access, on = 'globalid', how ='left', suffixes=('','_y'))
join.drop(join.filter(regex='_y$').columns, axis=1, inplace=True)
barriers = join
barriers_columnlist = list(barriers)

In [58]:
# join route_useage to barrier
join_left_df = barriers.sjoin_nearest(route_useage, how="left")
barriers_columnlist += ['total_pass']
barriers = join_left_df[barriers_columnlist]

In [59]:
betweeness = betweeness.to_crs(27700)

In [60]:
# join betweeness to barrier
join_left_df = barriers.sjoin_nearest(betweeness, how="left")
barriers_columnlist += ['Betweennes']
barriers = join_left_df[barriers_columnlist]

In [61]:
barriers

,globalid,AuditAmend,CriticalDi,OnNCN,Restrictio,Critical_1,Other,Status,CreationDa,Creator,...,geometry,origin_id,min_gain,CarPerCit,rank,objectid,amenity_diff,amenity_diff_bike,total_pass,Betweennes
0,11993240-4aa1-445a-a732-4aa1e8fd031b,auditing,0,yes,Chicane,113,None,Too Narrow,2021-11-09,Daniel.ebrahim_Sustrans_UK,...,POINT (195301.057 237047.905),11993240-4aa1-445a-a732-4aa1e8fd031b,14.410891,0.476,936,69205.0,0.0,1.0,253.0,614826.0
1,dcb3f04c-442b-4f29-8d63-8f58c71a8c63,auditing,0,yes,Gate,243,None,Acceptable Width,2021-11-09,Daniel.ebrahim_Sustrans_UK,...,POINT (195311.394 237045.757),dcb3f04c-442b-4f29-8d63-8f58c71a8c63,14.410891,0.476,936,69205.0,0.0,1.0,0.0,22838.0
2,dc4d7c69-3e1e-4312-a50d-e3970fa2cfde,auditing,0,yes,BollardMultiple,127,other,Too Narrow,2021-11-09,Daniel.ebrahim_Sustrans_UK,...,POINT (194786.639 236686.757),dc4d7c69-3e1e-4312-a50d-e3970fa2cfde,2.329219,0.476,936,34474.0,0.0,0.0,1582.0,3126096.0
3,37dd978e-bead-4fda-bc98-f412a0e53f2f,auditing,0,yes,Chicane,119,other,Too Narrow,2021-11-09,Daniel.ebrahim_Sustrans_UK,...,POINT (194787.344 236684.587),37dd978e-bead-4fda-bc98-f412a0e53f2f,2.329219,0.476,936,34474.0,0.0,0.0,1582.0,3126096.0
4,60f3e82a-6d2a-4920-8950-e5795e5bc09c,auditing,0,yes,Chicane,114,"slope,other",Too Narrow,2021-11-09,Daniel.ebrahim_Sustrans_UK,...,POINT (194720.893 236651.124),60f3e82a-6d2a-4920-8950-e5795e5bc09c,136.328799,0.476,936,34474.0,0.0,0.0,1582.0,3126874.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73,56b7113e-4439-4110-b1a1-66ab0eed6c56,auditing,0,yes,Gate,134,None,Too Narrow,2021-12-02,Cassia.Copeland_Sustrans_UK,...,POINT (218616.319 245004.753),56b7113e-4439-4110-b1a1-66ab0eed6c56,18862.328839,0.431,1148,NaN,0.0,0.0,0.0,22838.0
74,dfc8a00e-3fdd-4a62-81d6-8243c94e826e,auditing,0,yes,Gate,150,"surface,slope",Acceptable Width,2021-12-10,peter.taylor_Sustrans_UK,...,POINT (193974.622 214220.009),dfc8a00e-3fdd-4a62-81d6-8243c94e826e,223.947341,0.448,1435,NaN,0.0,0.0,91.0,9256669.0
75,32eb342c-c120-483e-8599-c44c6b3ca0ff,auditing,0,no,BollardMultiple,121,None,Not NCN,2021-12-10,peter.taylor_Sustrans_UK,...,POINT (176030.980 225288.146),32eb342c-c120-483e-8599-c44c6b3ca0ff,104.230977,0.462,1207,34328.0,1.0,6.0,0.0,22838.0
76,057a0715-4a3e-4479-a72d-fc78b0c300f0,Auditing,0,yes,Cattlegrid,342,None,Acceptable Width,2022-02-14,Cassia.Copeland_Sustrans_UK,...,POINT (211593.938 230663.127),057a0715-4a3e-4479-a72d-fc78b0c300f0,2467.644234,0.429,873,NaN,0.0,0.0,0.0,3663545.0


In [62]:
barriers.to_file(r"C:\Users\b8008458\OneDrive - Newcastle University\2021 to 2022\Dissertation\Github\BarrierRemoval\Study Areas\Pembrokeshire\Barriers\PembrokeBarriersMetrics.gpkg")